In [59]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Masking
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [60]:
import pose
from pose_parser import parse_file, detect_perspective
import numpy as np

In [61]:
good_videos = [parse_file("synthesized/bicep/bicep_good_"+ str(i) +".npy") for i in range(1,674)]
bad_videos = [parse_file("synthesized/bicep/bicep_bad_" + str(i) + ".npy") for i in range(1,524)]

pe:  (186, 18, 3)
Mean torso:  271.28176013118616
Data shape:  (128, 18, 3)
Mean torso:  271.313329797635
Data shape:  (92, 18, 3)
Mean torso:  271.47972418365396
Data shape:  (181, 18, 3)
Mean torso:  271.2683185518321
Data shape:  (128, 18, 3)
Mean torso:  271.313329797635
Data shape:  (93, 18, 3)
Mean torso:  271.16794805208
Data shape:  (183, 18, 3)
Mean torso:  271.43114525724275
Data shape:  (133, 18, 3)
Mean torso:  271.43121743197946
Data shape:  (99, 18, 3)
Mean torso:  271.4411855636567
Data shape:  (190, 18, 3)
Mean torso:  271.6414995496406
Data shape:  (133, 18, 3)
Mean torso:  271.43121743197946
Data shape:  (98, 18, 3)
Mean torso:  271.2296939254544
Data shape:  (199, 18, 3)
Mean torso:  271.31484601120764
Data shape:  (133, 18, 3)
Mean torso:  271.43121743197946
Data shape:  (89, 18, 3)
Mean torso:  270.8364063528552
Data shape:  (213, 18, 3)
Mean torso:  271.2307174331363
Data shape:  (133, 18, 3)
Mean torso:  271.43121743197946
Data shape:  (93, 18, 3)
Mean torso:  27

In [62]:
xs = []
ys = []
for video in good_videos + bad_videos:
    frames = video

    side = detect_perspective(frames)

    #Angles to calculate
    upperarm_forearm_angles = []
    upperarm_torso_angles = []
    sequence = []

    for frame in frames:
        if (side == pose.Side.right):
            upperarm = pose.Part(frame.relbow, frame.rshoulder)
            forearm = pose.Part(frame.relbow, frame.rwrist)
            torso = pose.Part(frame.rhip, frame.neck)
        else:
            upperarm = pose.Part(frame.lelbow, frame.lshoulder)
            forearm = pose.Part(frame.lelbow, frame.lwrist)
            torso = pose.Part(frame.lhip, frame.neck)
        angle1 = upperarm.calculate_angle(forearm)
        angle2 = upperarm.calculate_angle(torso)
        sequence.append(np.array([angle1, angle2]))
        upperarm_forearm_angles.append(angle1)
        upperarm_torso_angles.append(angle2)
    xs.append(np.array(sequence))
    if (video in good_videos):
        ys.append([0])
    else:
        ys.append([1])

Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primar

In [63]:
print(len(xs), len(ys))
print(ys)

1196 1196
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [64]:
#Convert to np array
xs = np.array(xs)
ys = np.array(ys)
print(xs.shape, ys.shape)
#Add padding
from keras.preprocessing import sequence
xs=sequence.pad_sequences(xs, padding='post', dtype='float', truncating='post')
ys=sequence.pad_sequences(ys, padding='post', dtype='float', truncating='post')
seq_len = xs.shape[1]
print(xs.shape, ys.shape)


(1196,) (1196, 1)
(1196, 262, 2) (1196, 1)


In [65]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=42)
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)


In [66]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 2)))
model.add(Dense(1, activation='sigmoid'))

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               265216    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 265,473
Trainable params: 265,473
Non-trainable params: 0
_________________________________________________________________


In [73]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=8, callbacks=[chk], validation_data=(X_test,y_test))
# model.predict(np.random.rand(10, 5, n_feats))   # This works

Train on 956 samples, validate on 240 samples
Epoch 1/3
956/956 [==============================] - 47s - loss: 0.3466 - acc: 0.8661 - val_loss: 0.2767 - val_acc: 0.9042
Epoch 2/3
956/956 [==============================] - 45s - loss: 0.3309 - acc: 0.8682 - val_loss: 0.2679 - val_acc: 0.9042
Epoch 3/3
956/956 [==============================] - 45s - loss: 0.3328 - acc: 0.8661 - val_loss: 0.2748 - val_acc: 0.9042


In [72]:
np.set_printoptions(suppress=True)
print(model.predict(X_test))
print(y_test)

[[0.15245718]
 [0.12324101]
 [0.15245408]
 [0.1524572 ]
 [0.1524572 ]
 [0.15245718]
 [0.99995005]
 [0.99995   ]
 [0.15245718]
 [0.15176472]
 [0.15245718]
 [0.99995005]
 [0.99995005]
 [0.9999501 ]
 [0.99995   ]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15241382]
 [0.1524572 ]
 [0.99995005]
 [0.1524572 ]
 [0.15245718]
 [0.15245718]
 [0.9999501 ]
 [0.15245718]
 [0.15245718]
 [0.09728861]
 [0.15245709]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.1524572 ]
 [0.99995637]
 [0.1524572 ]
 [0.1524572 ]
 [0.99995005]
 [0.99995637]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.9999565 ]
 [0.12323728]
 [0.9999509 ]
 [0.9999501 ]
 [0.1524572 ]
 [0.15245718]
 [0.15245718]
 [0.15245718]
 [0.15232024]
 [0.15245485]
 [0.1524572 ]
 [0.99995005]
 [0.1524572 ]
 [0.15158418]
 [0.15245718]
 [0.1524572 ]
 [0.9999564 ]
 [0.15245718]
 [0.15245718]
 [0.99995637]
 [0.99995005]
 [0.99995005]
 [0.15245718]
 [0.15099019]
 [0.9999501 ]
 [0.99995065]
 [0.15